In [2]:
from con_models import get_cons

In [8]:
con, use_colors, varname, pvarname, modname, warming_modname, hiresmd = get_cons()

In [7]:
use_colors[0]

'#88CCEE'

In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
import calendar
from global_land_mask import globe
import glob
import math
from scipy import stats
import os
import netCDF4 as nc

from highres_read import read_var_hires
from myReadGCMsDaily import read_var_mod
from regrid_wght_3d import regrid_wght_wnans

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100
plt.clf()
plt.rcParams['figure.figsize'] = (15.0/2.5, 15.0/2.5)

<Figure size 600x400 with 0 Axes>

In [41]:
#####Constants
Cp = 1004           #J/kg/K
Rd = 287            #J/kg/K
con= Rd/Cp

#latitude range
latr1 = 30
latr2 = 80

#pressure levels in observations
p_level_700 = 3  ### 700hPa

# use_colors = ['rosybrown','goldenrod','teal','blue','hotpink','green','red','cyan','magenta','cornflowerblue','mediumpurple','blueviolet',
# 'deeppink','lawngreen','coral','peru','salmon','burlywood','rosybrown','goldenrod','teal','blue','hotpink','green','red','cyan','magenta','yellow','cornflowerblue','mediumpurple','blueviolet',
# 'deeppink','lawngreen','coral','peru','salmon','burlywood']

use_colors = ['#88CCEE','#CC6677','#117733','#332288','#AA4499','#44AA99','#999933','#882255','#661100','#6699CC','#888888','#e6194b',
'#3cb44b','#0082c8','#f58231','#911eb4','#46f0f0','#f032e6','#d2f53c','#fabebe','#008080','#e6beff','#aa6e28','#fffac8','#800000','#aaffc3',
'#808000','#ffd8b1','#000080','#808080','#ffffff','#000000'] #'#ffe119', ,'#DDCC77'

highres_modname = ['FGOALS-f3-L','BCC-CSM2-HR','MRI-AGCM3-2-H','MRI-AGCM3-2-S'] 

varname = ['sfcWind', 'tas','psl'] #'sfcWind', 'hfss', 'hfls', 'tas', 'ps', 'psl',,'pr'
pvarname= ['ta']

In [37]:
time1=[2010, 1, 1]
time2=[2012, 12, 30]

In [43]:
l = 0
m = len(highres_modname)

In [ ]:
#read_var_hires(level, modn, exper, varnm, time1, time2)
for j in range(l,m):
    print(highres_modname[j])
    for i in varname:
        print(i)
        locals()[i+'__'+str(j+1)] = read_var_hires('surface', highres_modname[j], 'highresSST-present', i, time1, time2)

    for k in pvarname:
        print(k)
        locals()[k+'__'+str(j+1)] = read_var_hires('p_level', highres_modname[j], 'highresSST-present', k, time1, time2)

    print('done')

FGOALS-f3-L
sfcWind
length files: 1
tas
length files: 1
psl
length files: 1
ta
length files: 3
done
BCC-CSM2-HR
sfcWind
length files: 3
tas
length files: 4
psl
length files: 4
ta
length files: 16
done
MRI-AGCM3-2-H
sfcWind
length files: 1
tas
length files: 1
psl
length files: 1
ta
length files: 3
done
MRI-AGCM3-2-S
sfcWind
length files: 3
tas
length files: 3
psl
length files: 3
ta
length files: 38


In [10]:
time1=[2010, 1, 1]
time2=[2012, 12, 30]
try2 = read_var_mod('surface', 'CESM2', 'historical', 'sfcWind', time1, time2)

66


In [11]:
print(np.shape(try2[0]))
print(np.shape(try2[1]))
print(np.shape(try2[2]))
print(np.shape(try2[3]))
print(np.shape(try2[4]))

(96,)
(144,)
(1094, 3)
(0,)
(1094, 96, 144)


In [3]:
#latitude range
latr1 = 30
latr2 = 80

In [4]:
Cp = 1004           #J/kg/K
Rd = 287            #J/kg/K
con= Rd/Cp

In [5]:
modname = ['MRI-AGCM3-2-S']   #'NorESM2-LM','CanESM5','GFDL-CM4','CESM2-WACCM-FV2','FGOALS-f3-L',''INM-CM5-H'

varname = ['sfcWind', 'tas','psl'] #'sfcWind', 'hfss', 'hfls', 'tas', 'ps', 'psl',,'pr'
pvarname= ['ta']

#sfcWind_day_BCC-CSM2-HR_highresSST-present_r1i1p1f1_gn_20120301-20130228.nc
l = 0
m = len(modname)   #l+1

time1=[2010, 1, 1]
time2=[2012, 12, 30]

lats_edges = np.arange(latr1,latr2+1,5)
lons_edges = np.arange(-180,181,5)

#binning
n_bins  = 50
M_range = (-20,5)

for j in range(l,m):
    print(modname[j])
    for i in varname:
        locals()[i+'__'+str(j+1)] = read_var_hires('surface', modname[j], 'highresSST-present', i, time1, time2)

    for k in pvarname:
        locals()[k+'__'+str(j+1)] = read_var_hires('p_level', modname[j], 'highresSST-present', k, time1, time2)

MRI-AGCM3-2-S


TypeError: can only concatenate str (not "int") to str

In [67]:
M_plot = []
W_plot = []
W_erro = []
b_coun = []
m = len(modname)

for i in range(l,m):
    print(modname[i])
    for j in varname:
        lat  = locals()[j+'__'+str(i+1)][0]
        lon  = locals()[j+'__'+str(i+1)][1]
        time = locals()[j+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        print(j)
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        locals()['plot_'+j+str(i+1)] = np.array(np.multiply(maskm,locals()[j+str(i+1)][:,lat_ind1[0]:lat_ind2[0],:]))
        locals()['grid_'+j+str(i+1)] = regrid_wght_wnans(lats,lon,locals()['plot_'+j+str(i+1)],lats_edges,lons_edges)[0]



    for k in pvarname:
        print(k)
        lat  = locals()[k+'__'+str(i+1)][0]
        lon  = locals()[k+'__'+str(i+1)][1]
        time = locals()[k+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        locals()['plot_levels'+str(i+1)] = locals()['ta__'+str(i+1)][3]
        locals()['grid_'+k+str(i+1)] = []

        levels = locals()['plot_levels'+str(i+1)]

        for p in range(len(levels)):
            if levels[p] == 70000:
                print(levels[p])
                locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
                locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
                temp_700   = np.array(np.multiply(maskm,locals()[k+str(i+1)][:,p,lat_ind1[0]:lat_ind2[0],:]))
                grid_t_700 = regrid_wght_wnans(lats,lon,temp_700,lats_edges,lons_edges)[0]
                break;

    theta_700 = grid_t_700*(100000/70000)**con
    theta_t2m = locals()['grid_tas'+str(i+1)]*(100000/locals()['grid_psl'+str(i+1)])**con

    M_700  = theta_t2m - theta_700
#     plot_M = M_700.flatten()
#     plot_W = locals()['grid_sfcWind'+str(i+1)].flatten()

#     ind = np.argsort(plot_M)

#     final_M = np.sort(plot_M)
#     final_W = plot_W[ind]

#     indx = np.isnan(final_M*final_W)==False

    # bin_means, bin_edges, binnumber       = stats.binned_statistic(final_M[indx], final_W[indx], 'mean', bins=n_bins)
    # bin_means_c, bin_edges_c, binnumber_c = stats.binned_statistic(final_M[indx], final_W[indx], 'count',bins=n_bins)
    # bin_means_s, bin_edges_s, binnumber_s = stats.binned_statistic(final_M[indx], final_W[indx], 'std',  bins=n_bins)
    # bin_means_x, bin_edges_x, binnumber_x = stats.binned_statistic(final_M[indx], final_M[indx], 'mean', bins=n_bins)

    # ind_c = np.where(

MRI-AGCM3-2-S
sfcWind
tas
psl
ta
70000.0


ValueError: operands could not be broadcast together with shapes (1095,10,72) (1154,10,72) 

In [57]:
np.shape(grid_t_700)

(364, 10, 72)

In [38]:
modname = ['GFDL-CM4']
for j in range(l,m):
    print(modname[j])
    for i in varname:
        print(i)
        locals()[i+'__'+str(j+1)] = read_var_mod('surface', modname[j], 'historical', i, time1, time2)

    for k in pvarname:
        print(k)
        locals()[k+'__'+str(j+1)] = read_var_mod('p_level', modname[j], 'historical', k, time1, time2)

    print('done')

GFDL-CM4
sfcWind
tas
psl
done


In [48]:
M_plot = []
W_plot = []
W_erro = []
b_coun = []
m = len(modname)
for i in range(l,m):
    print(modname[i])

    for j in varname:
        lat  = locals()[j+'__'+str(i+1)][0]
        lon  = locals()[j+'__'+str(i+1)][1]
        time = locals()[j+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        print(j)
        locals()[j+str(i+1)] = locals()[j+'__'+str(i+1)][4]
        locals()[j+str(i+1)] = np.ma.filled(locals()[j+str(i+1)], fill_value=np.nan)
        locals()['plot_'+j+str(i+1)] = np.array(np.multiply(maskm,locals()[j+str(i+1)][:,lat_ind1[0]:lat_ind2[0],:]))
        locals()['grid_'+j+str(i+1)] = regrid_wght_wnans(lats,lon,locals()['plot_'+j+str(i+1)],lats_edges,lons_edges)[0]



    for k in pvarname:
        print(k)
        lat  = locals()[k+'__'+str(i+1)][0]
        lon  = locals()[k+'__'+str(i+1)][1]
        time = locals()[k+'__'+str(i+1)][2]

        x_lat = np.array(lat)
        lat_ind1 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr1)).argmin()])[0]
        lat_ind2 = np.where(x_lat == x_lat.flat[np.abs(x_lat - (latr2)).argmin()])[0]
        lats = lat[lat_ind1[0]:lat_ind2[0]]

        x_lon = lon
        lon = np.array(lon)
        lon[lon > 180] = lon[lon > 180]-360

        maskm = np.ones((len(time),len(lats),len(lon)))

        for a in range(len(lats)):
            for b in range(len(lon)):
                if globe.is_land(lats[a], lon[b])==True:
                    maskm[:,a,b] = math.nan
        locals()['plot_levels'+str(i+1)] = locals()['ta__'+str(i+1)][3]
        locals()['grid_'+k+str(i+1)] = []

        levels = locals()['plot_levels'+str(i+1)]

        for p in range(len(levels)):
            if levels[p] == 70000:
                print(levels[p])
                locals()[k+str(i+1)] = locals()[k+'__'+str(i+1)][4]
                locals()[k+str(i+1)] = np.ma.filled(locals()[k+str(i+1)], fill_value=np.nan)
                temp_700   = np.array(np.multiply(maskm,locals()[k+str(i+1)][:,p,lat_ind1[0]:lat_ind2[0],:]))
                grid_t_700 = regrid_wght_wnans(lats,lon,temp_700,lats_edges,lons_edges)[0]
                break;


GFDL-CM4
sfcWind


/glade/work/geethma/research/codes/regrid_wght_3d.py:46: RuntimeWarning: invalid value encountered in true_divide
  zout[i]=zz[0]/ww[0]


tas
psl
ta
70000.0


In [47]:
np.shape(ta1)

(1094, 8, 90, 144)

In [ ]:
fn     ='/glade/scratch/geethma/cmip6/p_level/'
name   = 'CMIP6.CMIP.NCAR.CESM2-WACCM.abrupt-4xCO2.r1i1p1f1.day.ta.gn.v20190425.ta_day_CESM2-WACCM_abrupt-4xCO2_r1i1p1f1_gn_01500101-01510101.nc'
ds     = nc.Dataset(fn+name)


In [ ]:
ds

In [ ]:
lons	= ds.variables['lon'][:]          #size=1800 #Range is 0 .. 358.75
lats	= ds.variables['lat'][:]       #size=400 #Range is -90...90
u		= ds.variables['ta'][:]    #size = 3650,192,288
time    = ds.variables['time'][:]
plev    = ds.variables['plev'][:]

In [ ]:
np.shape(plev)

In [ ]:
plev

In [ ]:
fn+name


In [ ]:
fn      ='/glade/collections/cmip/CMIP6/HighResMIP/CNRM-CERFACS/CNRM-CM6-1-HR/hist-1950/r1i1p1f2/day/pr/gr/v20190221/pr/pr_day_CNRM-CM6-1-HR_hist-1950_r1i1p1f2_gr_19500101-19541231.nc'
ds     = nc.Dataset(fn)
# ds.append(dsi)
lons	= ds.variables['lon'][:]          #size=1800 #Range is 0.1 .. 359.9
lats	= ds.variables['lat'][:]       #size=400 #Range is -39.9 .. 39.9
# u		= dsi.variables['wind_speed'][:,:,:]    #float wind_speed(time, lat, lon) #units = "m s-1"
# time    = ds.variables['time'][:]          #size=24
# level   = ds.variables['plev'][:]
# dsi.close

In [ ]:
lons[0:2]

In [ ]:
level[0]

In [ ]:
lons[-1]

In [ ]:
lats[42]

In [ ]:
lats[149]

In [ ]:
level

In [ ]:
np.shape(time)

In [ ]:
time[0]

In [ ]:
a =np.arange(9)
print(a)
b = np.reshape(a,(-1,9))

In [ ]:
print(b[0,0])

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.cm import get_cmap
import cartopy.crs as ccrs
import calendar
import glob

In [ ]:
from scipy import stats



In [ ]:

cao = []
sw  = []

fi = glob.glob('/glade/work/geethma/research/npzfilesn/cao/2016/M.npz')
fi = np.array(fi)
di = np.load(fi[0])
cao.extend(di['cao'])

fnn = glob.glob('/glade/work/geethma/research/npzfilesn/macsfcwind/2016/m*npz')
fnn = np.array(fnn)

for i in range(0,len(fnn)):
    dd = np.load(fnn[i])
    sw.extend(dd['sfcW'])

print(np.shape(cao))
print(np.shape(sw))






In [ ]:
d_r = []
for i in range(len(cao)):
    d_r.append(round(cao[i],1))

In [ ]:
cao_r = [np.round(x,1) for x in cao]

In [ ]:
sw_r = [np.round(x,1) for x in sw]

In [ ]:
cao_rr = cao_r[~np.isnan(cao_r)]

In [ ]:
ii = np.isfinite(cao_r[0])

In [ ]:
ind = []
for i in range(len(cao_r)):
    ii = np.isfinite(cao_r)
    ind.extend(ii)
print(np.shape(ind))

In [ ]:
print(ind[0:10])

In [ ]:
cao_r[0:10]

In [ ]:
fig = plt.figure()

plt.hexbin(cao_r[:], sw_r[:], gridsize=(15,15), cmap=plt.cm.Purples_r )
plt.xlabel('CAO [K]')
plt.ylabel('surface wind (m/s)')
plt.colorbar()

plt.show()



In [ ]:
c =[]
w =[]

for i in range(336):
    for j in range(101):
        for k in range(360):
            if  np.isnan(cao[i][j][k])==False & np.isnan(sw[i][j][k])==False:
                c.append(cao[i][j][k])
                w.append(sw[i][j][k])
bin_means, bin_edges, binnumber = stats.binned_statistic(c, w, statistic='mean')

fig = plt.figure()
plt.hlines(bin_means, bin_edges[:-1], bin_edges[1:], colors='g', lw=2,label='binned statistic of data')
plt.plot(c, x_pdf, 'r-', label='analytical pdf')
# plt.hist(w, bins=50, normed=True, histtype='stepfilled',alpha=0.2, label='histogram of data')
plt.show()

In [ ]:
x_pdf = stats.maxwell.pdf(c)

In [ ]:
plt.hist(w, bins=50, normed=True, histtype='stepfilled',alpha=0.2, label='histogram of data')